In [35]:
from __future__ import print_function
from builtins import range
import sys
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier

In [36]:

from gensim.models import KeyedVectors

In [37]:
train = pd.read_csv('/home/rkamath/fastai/train.txt',header=None,sep='\t')

In [38]:
test = pd.read_csv('/home/rkamath/fastai/test.txt',header=None, sep='\t')

In [39]:
train.columns = ['label','content']
test.columns = ['label','content']

In [40]:
import pdb
class GloveVectorizer():
    def __init__(self):
        print("Loading Vectors")
        word2vec = {}
        embedding = []
        idx2word = []
        with open('/home/rkamath/fastai/glove.txt') as f:
            for line in f:
                values = line.split()
                word = values[0]
                vec = np.asarray(values[1:],dtype="float32")
                word2vec[word] = vec
                embedding.append(vec)
                idx2word.append(word)
                
        print('Found %s word vectors' % len(word2vec))
    
        self.word2vec = word2vec
        self.embedding = np.array(embedding)
        
        self.word2idx = {v:k for k,v in enumerate(idx2word)}
        self.V ,self.D = self.embedding.shape
    def fit(self,data):
        pass
    
    def transform(self,data):
        
        X = np.zeros((len(data),self.D))
        
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.lower().split()
            vecs = []
            for word in tokens:
                if word in self.word2vec:
                    
                    vec = self.word2vec[word]
                    vecs.append(vec)
            if len(vecs)>0:
                
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis = 0)

            else:
                emptycount += 1
            n+=1
        print("No of sammples with no word found: %s %s "%(emptycount, len(data)))
        return X
    
    def fit_transform(self,data):
        self.fit(data)
        return self.transform(data)

In [41]:
vectorizer = GloveVectorizer()
Xtrain = vectorizer.fit_transform(train.content)
Ytrain = train.label

Loading Vectors
Found 400000 word vectors
No of sammples with no word found: 0 11293 


In [43]:
class Word2VecVectorizer:
    def __init__(self):
        self.word_vectors = KeyedVectors.load_word2vec_format('/home/rkamath/fastai/GoogleNews-vectors-negative300.bin',binary=True)
        print("Finished loading the word vectors")
    
    def fit(self,data):
        pass
    
    def tranform(self,data):
        v = self.word_vectors.get_vector('king')
        self.D = v.shape[0]
        
        X = np.zeros((len(data),self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.lower().split()
            vecs = []
            m = 0
            for words in tokens:
                try:
                    vec = self.word_vectors.get_vector(words)
                    vecs.append(vec)
                    m+=1
                except KeyError:
                    pass
            if len(vecs)>0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis =0)
            else:
                emptycount += 1
            n+=1
        print("No of sample with no word found : %s %s"%(emptycount,len(data)))
        return X
        
    def fit_transform(self,data):
        self.fit(data)
        return self.tranform(data)
        
        

In [44]:
vectorizer = Word2VecVectorizer()

Finished loading the word vectors


In [45]:
Xtrain = vectorizer.fit_transform(train.content)
Ytrain = train.label

No of sample with no word found : 0 11293
